In [ ]:
####
# This notebook imports the following data sources:
#  - gnomAD v2.1 hail table
#  - ClinVar GRCh37 VCF
# The update to gnomad v3 presented significant changes to the schema of metadata which 
# breaks the handling of AF and populations in this notebook.
# It also renamed the contigs in the ref column which breaks the join with clinvar.
#
"""
ClinVar:
- clinvar_variation_id
- clinvar_review_status
- clinvar_significance
- clinvar_significance_interpretations
GnomAD:
- faf95
- source
- popmax_pop
- popmax_ac
- popmax_an
- genomic_coordinates
- hgvs
- protein_effect
"""

# Set the output bucket to write to, dataproc service account must have write access
# Do not include trailing slash or "gs://"
output_bucket = ""
assert(len(output_bucket) > 0)
# Set the TSV path to write into bucket. Can contain slash like "folder/file.tsv"
# Do not include leading slash
output_filename = "report.tsv"

reference_genome = "GRCh37"

In [ ]:
import hail as hl
# `idempontent=True` is useful for running all cells in the notebook
hl.init(idempotent=True)

In [ ]:
# utility functions for file placement
import subprocess
import os
import time, datetime

def run_args(args, fail_on_stderr=False, success_codes=[0]) -> tuple: # (stdout,stderr,returncode)
    print(args)
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = p.communicate()
    if (fail_on_stderr and len(stderr) > 0) or (p.returncode not in success_codes):
        raise RuntimeError("command {} failed with code {}:{}".format(
            args, p.returncode, stderr))
    return (stdout, stderr, p.returncode)

def local_to_bucket(local_path:str, gcs_path:str):
    if not gcs_path.startswith("gs://"):
        gcs_path = "gs://{}/{}".format(output_bucket, gcs_path)
    args = ["gsutil", "cp", local_path, gcs_path]
    run_args(args)
    
def bucket_to_local(gcs_path:str, local_path:str):
    if not gcs_path.startswith("gs://"):
        gcs_path = "gs://{}/{}".format(output_bucket, gcs_path)
    args = ["gsutil", "cp", gcs_path, local_path]
    run_args(args)
    
def local_to_hdfs(local_path:str, hdfs_path:str):
    if not local_path.startswith("/"):
        local_path = os.path.join(os.getcwd(), local_path)
    args = ["hdfs", "dfs", "-rm", hdfs_path]
    run_args(args, success_codes=[0,1]) # Allow error
    args = ["hadoop", "fs", "-cp", "file://" + local_path, hdfs_path]
#     args = ["hdfs", "dfs", "-cp", "file://" + local_path, hdfs_path]
    run_args(args)
    
def hdfs_to_local(hdfs_path:str, local_path:str):
    if os.path.exists(local_path):
        os.remove(local_path)
    args = ["hdfs", "dfs", "-cp", hdfs_path, "file://" + local_path]
    run_args(args)

In [ ]:
import io, re



# thresh_reader = io.StringIO(thresholds)

        
# gene_thresholds, gene_refseqs = parse_thresholds(thresh_reader)

# print("gene_thresholds:\n%s" % gene_thresholds)
# print("input refseqs:\n%s" % gene_refseqs)

# gene_thresholds_exp = hl.literal(gene_thresholds)


In [ ]:
import io
import re

# Read gnomAD data as Hail Tables
# ds_exomes = hl.read_table("gs://gnomad-public/release/2.1.1/ht/exomes/gnomad.exomes.r2.1.1.sites.ht")
# ds_exomes = hl.read_table(
#     "gs://gnomad-public-requester-pays/release/2.1.1/ht/exomes/gnomad.exomes.r2.1.1.sites.ht")

# ds_exomes = ds_exomes.annotate(
#     source="gnomAD Exomes"
# )
# ds_genomes = hl.read_table("gs://gnomad-public/release/2.1.1/ht/genomes/gnomad.genomes.r2.1.1.sites.ht")
ds_genomes = hl.read_table(
    "gs://gnomad-public-requester-pays/release/2.1.1/ht/genomes/gnomad.genomes.r2.1.1.sites.ht")
ds_genomes = ds_genomes.annotate(
    source="gnomAD Genomes"
)

# Can perform a union here if wanting both (ds = ds1.union(ds2))
def select_necessary_cols(ds):
    ds = ds.select(ds.freq, ds.faf, ds.vep, ds.source)
    return ds

# ds_exomes = select_necessary_cols(ds_exomes)
ds_genomes = select_necessary_cols(ds_genomes)
ds = ds_genomes
# ds = ds_genomes.union(ds_exomes, unify=True)

# Show the schema of the hail Table
ds.describe()

In [ ]:
# START DEBUGGING CELLS

In [ ]:
import json
ds.describe()
print(ds.globals.collect())
testds = ds.filter(ds.locus==hl.locus("16", 2185899))
testds.show()
print(testds.collect())


In [ ]:
# END DEBUGGING CELLS

In [ ]:
"""
ds.freq has raw frequency information, including AN, AC, and pop label. This is an array of 
structs, at indices determined by the categories in ds.globals.freq_index_dict

ds.faf has filtered allele frequency information, including confidence intervals faf95 adn faf99.
This is an array of structs, at indices determined by the category map in ds.globals.faf_index_dict
"""

def add_popmax_af(ds):
    """
    Adds a popmax_faf and popmax_af_pop column to the ds Hail Table.
    
    popmax_faf is a faf structure from the original ds, containing the maximum faf of the
    listed faf structures in the original ds, based on the filtering criteria 
    `default_faf_filter_type`. 
    
    The popmax_index_dict_key column contains the text field from the
    ds.globals.faf_index_dict which corresponds to each popmax_faf. This is similar to the
    ds.popmax_faf.meta["pop"] value but not exactly the same (gnomad_afr vs afr)
    
    Returns the updated ds.
    """
    # Identify indices in FAF field that correspond to the entire dataset (not a subset like non-cancer)
    # faf_index_map = [(k,v) for k, v in hl.eval(ds.globals.faf_index_dict).items() if k.startswith("gnomad_")]
    from enum import Enum
    class FafFilterType(Enum):
        # Each correponds to a filter func for a (k,v) of faf label to value
        GNOMAD_GLOBAL = lambda t: t[0] == "gnomad"
        GNOMAD_SUPERPOP = lambda t: t[0].startswith("gnomad_")
        ANY = lambda t: True

    # By default, filter to superpopulations aggregate faf
    default_faf_filter_type = FafFilterType.GNOMAD_SUPERPOP

    def faf_filter(faf_idx_tuple:tuple):
        return default_faf_filter_type(faf_idx_tuple)

    # Get list of the global faf_index_dict which meets the default_faf_filter criteria
    # This gives the indices of the desired populations, by default will take all top level populations
    faf_index_map = list(filter(faf_filter, [(k,v) for k,v in hl.eval(ds.globals.faf_index_dict).items()]))
    print("faf_index_map:\n" + str(faf_index_map))
    faf_indices = [v for k,v in faf_index_map]
    faf_labels = [k for k,v in faf_index_map]
    
    # Annotate table with popmax FAF
    
    # This only will return the maximum pop FAF for each
    # variant, even if multiple populations meet the criteria. 
    # If we want all matching populations, need an explode() call
    # to flatten the pop FAFs into a record per pop per variant
    
    ds = ds.annotate(
        popmax_faf=hl.sorted(
            # Take only the FAF entries that correspond to the desired populations (faf_indices)
            hl.literal(faf_indices).map(lambda i: ds.faf[i]),
            # Sort by 95% confidence FAF
            lambda faf_entry: faf_entry.faf95,
            # Sort high to low
            reverse=True
        )[0] # Take the first entry with the highest FAF
        ,
        # Label of the freq_index_dict entry for this record's max pop
        popmax_index_dict_key=hl.sorted(
            # List of tuples of (poplabel, faf_index)
            list(zip(list(faf_labels), list(faf_indices))),

            # Take only the FAF entries that correspond to the entire dataset
            # Sort by 95% confidence FAF
            key=lambda tpl: ds.faf[tpl[1]].faf95,
            # Sort high to low
            reverse=True
        )[0][0] # Take the first entry's label, which has the highest FAF
    )
    
    ds = ds.annotate(
#         popmax_faf_pop_freq=ds.freq[ds.globals.freq_index_dict["gnomad_" + ds.popmax_faf.meta.get("pop")]]

        # ds.globals.freq_index_dict uses the same keys as ds.globals.faf_index_dict so
        # we can reuse ds.popmax_index_dict_key created above
        popmax_faf_pop_freq=ds.freq[ds.globals.freq_index_dict[ds.popmax_index_dict_key]] 
    )
    
    return ds


ds_with_popmax = add_popmax_af(ds)

In [ ]:
ds_with_popmax.describe()
# ds_with_popmax.show()
t = ds_with_popmax
t = t.select(t.faf, t.popmax_faf, t.popmax_index_dict_key, t.popmax_faf_pop_freq)
L = t.take(10)
print(L[3])

In [ ]:

ds_formatted = ds_with_popmax.select(
    source = ds_with_popmax.source,
#     gene = ds.gene,
    popmax_pop = ds_with_popmax.popmax_faf.meta["pop"],
    popmax_ac = ds_with_popmax.popmax_faf_pop_freq.AC,
    popmax_an = ds_with_popmax.popmax_faf_pop_freq.AN,
    faf95 = ds_with_popmax.popmax_faf.faf95,
    genomic_coordinates = hl.format("%s-%s-%s-%s",
        ds_with_popmax.locus.contig,
        hl.str(ds_with_popmax.locus.position),
        ds_with_popmax.alleles[0],
        ds_with_popmax.alleles[1]
    )
)
ds_formatted.filter(ds_formatted.popmax_ac != 0).show()

In [ ]:
# clinvar = hl.import_vcf("/path/to/clinvar.vcf.gz", force_bgz=True, drop_samples=True, skip_invalid_loci=True).rows()

# Download clinvar BGZF
import os, requests, subprocess

# Function to download a file to a localpath.
def download_to_file(url, filepath):
    r = requests.get(url, stream=True)
    with open(filepath, "wb") as fout: 
        for chunk in r.iter_content(chunk_size=1024): 
             if chunk:
                 fout.write(chunk)


# This url always points to the latest dump file, updated periodically by ClinVar
clinvar_vcf_url = "https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz"
clinvar_vcf_localpath = "/home/hail/clinvar.vcf.gz"
clinvar_vcf_hdfs = "clinvar.vcf.gz"
# ClinVar VCF is small enough to download to dataproc default local disk.
download_to_file(clinvar_vcf_url, clinvar_vcf_localpath)
assert(os.path.exists(clinvar_vcf_localpath))
print("Downloaded ClinVar VCF, file size (expecting ~28M): %d" % os.path.getsize(clinvar_vcf_localpath))

# Hail needs the file in HDFS
# def local_to_hdfs(local_path:str, hdfs_path:str):

local_to_hdfs(clinvar_vcf_localpath, clinvar_vcf_hdfs)


In [ ]:
# Load the ClinVar VCF into Hail Table
clinvar = hl.import_vcf(
    clinvar_vcf_hdfs,
    force_bgz=True,
    drop_samples=True, 
    skip_invalid_loci=True
).rows()
print("ClinVar row count: " + str(clinvar.count()))

# Join ClinVar table to gnomAD table. ClinVar fields available under the table.clinvar struct
ds_gnomad_clinvar = ds_formatted.annotate(
    clinvar=clinvar[ds_formatted.locus, ds_formatted.alleles]
)

ds_gnomad_clinvar.show()

# ClinVar VCF export sets ID column to the ClinVar Variation ID (not rsid)
# And sets the RS field of INFO to the rsid if it exists.
# (https://ftp.ncbi.nlm.nih.gov/pub/clinvar/README_VCF.txt)
# Hail then sets this ClinVar ID as the rsid column of the clinvar struct
# We can filter to only the variants that exist in clinvar with:
# gnomad_clinvar_ds = gnomad_clinvar_ds.filter(
#     ~hl.is_missing(gnomad_clinvar_ds.clinvar_rsid)
# )

In [ ]:
# Filter ds_gnomad_clinvar to those in input set
ds_gnomad_clinvar.filter(~hl.is_missing(ds_gnomad_clinvar.clinvar.info.CLNSIG)).show()


In [ ]:
# Load input file to join to
filename = "invitae-variant-input.tsv"
bucket_to_local(filename, filename)
local_to_hdfs(filename, filename)
variant_input = hl.import_table(filename, delimiter='\t')
# ds_filtered = ds_gnomad_clinvar.filter(
#     hl.any() #TODO
# )
variant_input.show()

In [ ]:
# Left join gnomad+clinvar to the input table
# ClinVar HGVS is in clinvar.info.CLNHGVS string array
c = clinvar.annotate(hgvs=clinvar.info.CLNHGVS)
c = c.explode(c.hgvs)
c = c.key_by(c.hgvs)

variant_input = variant_input.key_by(variant_input.Chromosomal_Variant)
ds_joined = variant_input.annotate(
    clinvar=c[variant_input.Chromosomal_Variant]
)


In [ ]:
# Debugging
ds_clinvar_missing = ds_joined.filter(hl.is_missing(ds_joined.clinvar))
print("ClinVar missing from {} input variants".format(ds_clinvar_missing.count()))



In [ ]:
ds_joined2 = ds_joined.key_by(ds_joined.clinvar.locus, ds_joined.clinvar.alleles)
gnomad = ds_formatted
ds_joined2 = ds_joined2.annotate(
    gnomad=gnomad[ds_joined2.clinvar.locus, ds_joined2.clinvar.alleles]
)
ds_joined2.show()

In [ ]:
# Select desired output fields (columns are ordered as provided)
output_ds = ds_joined2
output_ds = output_ds.select(
    output_ds.Input_Variant,
    output_ds.Chromosomal_Variant,
    gnomad_source=output_ds.gnomad.source,
    gnomad_faf95=output_ds.gnomad.faf95,
    gnomad_popmax_pop=output_ds.gnomad.popmax_pop,
    gnomad_popmax_ac=output_ds.gnomad.popmax_ac,
    gnomad_popmax_an=output_ds.gnomad.popmax_an,
    gnomad_genomic_coordinates=output_ds.gnomad.genomic_coordinates,
    clinvar_variation_id=output_ds.clinvar.rsid,
    clinvar_review_status=hl.delimit(output_ds.clinvar.info["CLNREVSTAT"], ","),
    clinvar_significance=hl.delimit(output_ds.clinvar.info["CLNSIG"], ","),
    clinvar_significance_interpretations=hl.delimit(output_ds.clinvar.info["CLNSIGCONF"], ",")
)

output_ds.describe()

# Export to TSV
report_filename = "report.tsv"
import time
print("Starting export to %s" % report_filename)
start_time = time.time()
output_ds.export(report_filename)
end_time = time.time()
print("Export took %.2f seconds" % (end_time - start_time))

In [ ]:
# The export is in HDFS now, copy to machine-local file
report_localpath = os.path.join(os.getcwd(), report_filename)
os.system("rm %s" % report_localpath)
hdfs_to_local(report_filename, report_localpath)


In [ ]:
# Upload to bucket and filepath set at top of notebook
gs_output_file = "gs://%s/%s" % (output_bucket, output_filename)
local_to_bucket(report_localpath, gs_output_file)
